## Модель BERT и GPT

Задание

Взять датасет
https://huggingface.co/datasets/merionum/ru_paraphraser
решить задачу парафраза

(дополнительно необязательная задача)на выбор взять
https://huggingface.co/datasets/sberquad
https://huggingface.co/datasets/blinoff/medical_qa_ru_data
натренировать любую модель для вопросно ответной системы
как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER

In [1]:
!pip install datasets
!pip install transformers
!pip install torchmetrics
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 29.7 MB/s 
     |████████████████████████████████| 115 kB 75.6 MB/s 
     |████████████████████████████████| 212 kB 19.1 MB/s 
     |████████████████████████████████| 141 kB 64.4 MB/s 
     |████████████████████████████████| 101 kB 16.1 MB/s 
     |████████████████████████████████| 596 kB 60.6 MB/s 
     |████████████████████████████████| 127 kB 66.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 15.6 MB/s 
     |████████████████████████

In [5]:
import pandas as pd

In [2]:
from datasets import load_dataset

corpus_train = load_dataset('merionum/ru_paraphraser', data_files='train.jsonl')
corpus_test = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-a81eaa3390aef201/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-57cfc8af008bb08a/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
corpus_train, corpus_test

(DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
}),
 DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
}))

In [6]:
df_train = pd.DataFrame(corpus_train['train'])
df_train['text'] =  list(zip(df_train['text_1'], df_train['text_2']))
df_train = df_train[:2000]
df_train.head()

,id,id_1,id_2,text_1,text_2,class,text
0,1,201,8159,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,(Полицейским разрешат стрелять на поражение по...
1,2,202,8158,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,(Право полицейских на проникновение в жилище р...
2,3,273,8167,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,(Президент Египта ввел чрезвычайное положение ...
3,4,220,8160,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,(Вернувшихся из Сирии россиян волнует вопрос т...
4,5,223,8160,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,(В Москву из Сирии вернулись 2 самолета МЧС с ...


In [7]:
#Классы: 1 -- точные перефразировки, 0 -- почти парафразы, -1 -- непарафразы
df_train['class'].value_counts()

0     745
1     642
-1    613
Name: class, dtype: int64

In [8]:
df_test = pd.DataFrame(corpus_test['train'])
df_test['text'] =  list(zip(df_test['text_1'], df_test['text_2']))
df_test = df_test[:500]
df_test.head()

,id,id_1,id_2,text_1,text_2,class,text
0,25349,34420,34421,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,-1,"(Цены на нефть восстанавливаются, Парламент Сл..."
1,25360,34438,34439,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,-1,"(""Гоголь-центр"" покажет видеозапись скандально..."
2,25382,34475,34476,Агент: РФС вновь задерживает зарплату Фабио Ка...,СМИ: Агент Фабио Капелло грозится подать в суд...,-1,(Агент: РФС вновь задерживает зарплату Фабио К...
3,25423,34494,34519,День Победы в Москве обещает выдаться облачным,Любляна отпразднует День Победы вместе с Москвой,-1,(День Победы в Москве обещает выдаться облачны...
4,25430,34531,34532,Посол РФ в США: Россия будет бороться с попытк...,Правительство запланировало заработать на лоте...,-1,(Посол РФ в США: Россия будет бороться с попыт...


In [9]:
#Классы: 1 -- точные перефразировки, 0 -- почти парафразы, -1 -- непарафразы
df_test['class'].value_counts()

-1    234
0     151
1     115
Name: class, dtype: int64

In [10]:
#LabelEncoder
y_train = df_train['class'].values
y_test = df_test['class'].values

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)

label_list = le.classes_  #array(['-1', '0', '1'], dtype=object)
label_list

y_train = le.transform(y_train) #array([1, 1, 1, ..., 0, 0, 0])
y_test = le.transform(y_test)
#le.inverse_transform(y_train) #array(['0', '0', '0', ..., '-1', '-1', '-1'], dtype=object)

In [11]:
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer

model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

tokenizer = BertTokenizer.from_pretrained(model_name)

In [12]:
#Создаем датасеты
class ParaphraseDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, classes):
        self._classes = classes

        self.tokenizer = tokenizer
        #для каждого text возвращает батч с полями:
               #'inputs_ids' -- тензор размера (B,1,max_len) из id токенов
               #'token_type_ids' -- тензор размера (B,1,max_len) из id типов токенов
               #'attention_mask' -- тензор размера (B,1,max_len) из индексов, указывающих, на какие токеты модель должна обратить внима
        self._txts = txts
                
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._classes[index]

In [13]:
train_dataset = ParaphraseDataset(df_train['text'], y_train)
test_dataset = ParaphraseDataset(df_test['text'], y_test)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=4,
                          shuffle=True,
                          num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset,
                          batch_size=4,
                          shuffle=False,
                          num_workers=0)

In [14]:
for txt, cls in train_loader:
    print(txt)
    print(txt[0])
    print(txt[1])
    print(len(txt[0]))
    print(cls)
    break

[('"Дочку" Газпрома обвинили в причастности к уклонению от налогов.', 'Право полицейских на проникновение в жилище решили ограничить.', 'Кехман рассказал об обысках в Михайловском театре и деле JFC.', 'В Кузбассе нашли тело одного из 4 пропавших в шахте горняков.'), ('Полиция уличила «дочку» «Газпрома» в уклонении от уплаты налогов.', 'Госдуме предложили ограничить полицию в праве проникать в жилище.', 'Михайловский театр обыскали по делу о 10-миллиардном мошенничестве.', 'В шахте в Кузбассе нашли тело одного из пропавших горняков.')]
('"Дочку" Газпрома обвинили в причастности к уклонению от налогов.', 'Право полицейских на проникновение в жилище решили ограничить.', 'Кехман рассказал об обысках в Михайловском театре и деле JFC.', 'В Кузбассе нашли тело одного из 4 пропавших в шахте горняков.')
('Полиция уличила «дочку» «Газпрома» в уклонении от уплаты налогов.', 'Госдуме предложили ограничить полицию в праве проникать в жилище.', 'Михайловский театр обыскали по делу о 10-миллиардном м

Используем предобученную модель cointegrated/rubert-base-cased-dp-paraphrase-detection (два класса: 0 --non-paraphrase, 1 -- paraphrase)

In [15]:
#загрузка модели
model = AutoModelForSequenceClassification.from_pretrained(model_name)
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [16]:
#пример токенизации
text1 = df_test['text'][0][0]
text2 = df_test['text'][0][1]
batch = tokenizer(text1, text2, return_tensors='pt')
batch

{'input_ids': tensor([[  101, 59713,  1469, 20264, 26631,  5052,   102, 28362, 27443, 57650,
         37777, 12678,  5576,  1758, 50938,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [17]:
#пример работы
def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt')
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1)
    return proba[0] # 0(non-paraphrase), 1(paraphrase)

text1 = df_test['text'][6][0]
text2 = df_test['text'][6][1]
proba = compare_texts(text1,text2)
cl = torch.argmax(proba)

print(text1)
print(text2)
print(proba)
print('predicted class: ', cl.numpy())
print('real class: ', le.inverse_transform([y_test[6]]))

Самолет вернулся в аэропорт Новосибирска из-за стука в салоне
Самолет вернулся в новосибирский аэропорт из-за таинственного стука
tensor([0.0231, 0.9769])
predicted class:  1
real class:  ['1']


In [18]:
#изменяем количество выходов в последнем слое классификации
import torch
import torch.nn as nn

model.classifier = nn.Linear(in_features=768, out_features=3, bias=True)
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [19]:
import torchmetrics
test_f1 = torchmetrics.F1Score()

for test_input, test_class in test_loader:
    with torch.inference_mode():
        batch = tokenizer(test_input[0], test_input[1], padding='max_length', max_length=64, truncation=True, return_tensors="pt")
        proba = torch.softmax(model(**batch).logits, -1) #torch.Size([B, 3])
        preds = torch.argmax(proba,dim=1) #torch.Size([B])
        target = test_class #torch.Size([B])
        
        test_f1(preds, target)

print(f'test f1_score: {test_f1.compute().item():.3f}')

test f1_score: 0.428


In [20]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [21]:
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.classifier.parameters()]))

Parameters full train: 177855747
Parameters transfer learning: 2307


In [22]:
#компиляция модели
criterion = nn.CrossEntropyLoss()
# обучаем последний слой классификации
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)  

In [23]:
#дообучение модели и подсчет метрик на тренировочном и валидационном датасетах
import torch
train_f1 = torchmetrics.F1Score()
test_f1 = torchmetrics.F1Score()

epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss_train = 0.0
    for train_input, train_class in train_loader:
        batch = tokenizer(train_input[0], train_input[1], padding='max_length', max_length=64, truncation=True, return_tensors="pt")
        proba = torch.softmax(model(**batch).logits, -1) #torch.Size([B, 3])
        target = train_class  # torch.Size([B])
        
        #loss
        batch_loss = criterion(proba, target)
        total_loss_train += batch_loss.item()

        #f1
        preds = torch.argmax(proba,dim=1) #torch.Size([B]))
        train_f1(preds, target)
        
        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_test = 0.0
    for test_input, test_class in test_loader:
        batch = tokenizer(test_input[0], test_input[1], padding='max_length', max_length=64, truncation=True, return_tensors="pt")
        proba = torch.softmax(model(**batch).logits, -1) #torch.Size([B, 3])
        target = test_class  # torch.Size([B])
        
        #loss
        batch_loss = criterion(proba, target)
        total_loss_test += batch_loss.item()
       
       #f1
        preds = torch.argmax(proba,dim=1) #torch.Size([B]))
        test_f1(preds, target)
            
    print(
        f'Epochs: {epoch + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train f1: {train_f1.compute().item(): .3f} \
        | Test Loss: {total_loss_test / len(test_dataset): .3f} \
        | Test f1: {test_f1.compute().item(): .3f}')
    train_f1.reset()
    test_f1.reset()

Epochs: 1 | Train Loss:  0.218         | Train f1:  0.647         | Test Loss:  0.219         | Test f1:  0.662
Epochs: 2 | Train Loss:  0.212         | Train f1:  0.690         | Test Loss:  0.219         | Test f1:  0.662
Epochs: 3 | Train Loss:  0.210         | Train f1:  0.698         | Test Loss:  0.216         | Test f1:  0.670
Epochs: 4 | Train Loss:  0.211         | Train f1:  0.692         | Test Loss:  0.213         | Test f1:  0.672
Epochs: 5 | Train Loss:  0.208         | Train f1:  0.715         | Test Loss:  0.215         | Test f1:  0.674
Epochs: 6 | Train Loss:  0.209         | Train f1:  0.703         | Test Loss:  0.212         | Test f1:  0.694
Epochs: 7 | Train Loss:  0.207         | Train f1:  0.709         | Test Loss:  0.214         | Test f1:  0.682
Epochs: 8 | Train Loss:  0.208         | Train f1:  0.696         | Test Loss:  0.217         | Test f1:  0.678
Epochs: 9 | Train Loss:  0.209         | Train f1:  0.698         | Test Loss:  0.215         | Test f1:

In [24]:
#метрика дообученной модели на валидационном датасете
test_f1 = torchmetrics.F1Score()
model.eval()

for test_input, test_class in test_loader:
    batch = tokenizer(test_input[0], test_input[1], padding='max_length', max_length=64, truncation=True, return_tensors="pt")
    proba = torch.softmax(model(**batch).logits, -1) #torch.Size([B, 3])
    preds = torch.argmax(proba,dim=1) #torch.Size([B])
    target = test_class #torch.Size([B])
        
    test_f1(preds, target)

print(f'test f1_score: {test_f1.compute().item():.3f}')

test f1_score: 0.696
